#### **Reporte Caso 2** ####
#### **Diseño Óptimo de un Reactor QuÍmico** ####
#### **Asignatura:** Programación para Analítica Prescriptiva y de Apoyo a la Decisión ####
#### **Instructor:** Dr. Josué Domínguez Guerrero ####
#### **Alumno:** Aminadab Córdova Acosta ####

**1. Definición del problema**

En los procesos químicos industriales, el diseño adecuado de un reactor es fundamental para asegurar una conversión eficiente de los reactivos en productos. La eficiencia de estos procesos está estrechamente relacionada con parámetros operativos como la temperatura del reactor y su volumen.
En este ejercicio se busca optimizar el diseño de un reactor de flujo pistón continuo (PFR, por sus siglas en inglés), en el cual se lleva a cabo una reacción de primer orden:

A → B

La velocidad de reacción está gobernada por una cinética de primer orden, y la constante de velocidad sigue una ley de Arrhenius, lo que introduce una relación no lineal con respecto a la temperatura.

El objetivo es encontrar la combinación óptima de temperatura y volumen del reactor que maximice la conversión del reactivo A, dentro de límites operacionales razonables.

---

Datos del problema

| Parámetro | Valor | Descripción |
|----------|-------|-------------|
| $C_{A0}$ | 1.0 mol/L | Concentración inicial de A |
| $F_{A0}$ | 10 mol/min | Flujo molar de entrada de A |
| $k_0$ | $1.0 \times 10^6$ 1/min | Factor pre-exponencial |
| $E$ | 8000 cal/mol | Energía de activación |
| $R$ | 1.987 cal/mol·K | Constante de los gases |
| $T$ | Variable (300–500 K) | Temperatura |
| $V$ | Variable (0–10 m³) | Volumen del reactor |

---

**2. Modelo Matemático**

Modelo Matemático

1. Constante de velocidad de reacción (Ley de Arrhenius):

$$
k = k_0 \cdot \exp\left(-\frac{E}{R T}\right)
$$

2. Conversión fraccional del reactivo:

$$
X = 1 - \exp\left(-\frac{k \cdot V}{F_{A0}}\right)
$$

3. Función objetivo:

Maximizar $X$, es decir:

$$
\max_{T, V} \left[ 1 - \exp\left( -\frac{k_0 \cdot \exp\left(-\frac{E}{R T}\right) \cdot V}{F_{A0}} \right) \right]
$$

4. Restricciones:

$$
300 \leq T \leq 500 \\
0 \leq V \leq 10
$$

---

**3. Implementación en Pyomo (Optimización No Lineal)**

In [2]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Crear modelo concreto
model = ConcreteModel()

# Parámetros
C_A0 = 1.0       # mol/L (no se usa directamente)
F_A0 = 10.0      # mol/min
k0 = 1.0e6       # 1/min
E = 8000.0       # cal/mol
R = 1.987        # cal/mol·K

# Variables de decisión
model.T = Var(bounds=(300, 500), within=Reals)   # Temperatura en K
model.V = Var(bounds=(0, 10), within=Reals)      # Volumen en m³

# Expresión para la constante de velocidad k (ley de Arrhenius)
def k_rule(model):
    return k0 * exp(-E / (R * model.T))
model.k = Expression(rule=k_rule)

# Expresión para la conversión X
def conversion_rule(model):
    return 1 - exp(-model.k * model.V / F_A0)
model.X = Expression(rule=conversion_rule)

# Función objetivo: maximizar la conversión X
model.obj = Objective(expr=model.X, sense=maximize)

# Resolver
solver = SolverFactory('ipopt')  # Asegúrate de tener IPOPT instalado
result = solver.solve(model, tee=True)

# Mostrar resultados
print(f"Temperatura óptima (T): {model.T():.2f} K")
print(f"Volumen óptimo (V): {model.V():.2f} m³")
print(f"Conversión máxima (X): {model.X():.4f}")


Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Tot

**4. Interpretación de la decisión preescrita por el modelo de decisión**

El modelo recomienda operar el reactor a una temperatura de 387.5 K y un volumen de 6.4 m³ para lograr una conversión máxima del 100% del reactivo A a producto B. Esta configuración garantiza la mayor eficiencia posible dentro de los límites operacionales establecidos.
